---
title: Tutorial - Adding Data to Graphs
author: "Shackett"
date: "May 9th 2025"
---

This notebook describes how we can add species- or reaction-level data to pathway representation (`SBML_dfs`), and also how we can propagate these attributes to the vertices and edges in a `napistu_graph`.

## Adding data to pathways

Species- and reaction-level data is associated with the `species_data` or `reactions_data` attribute of an `SBML_dfs` object. Each of these fields can include multiple sources of entity data organized as a dictionary where keys are an information source label, and values are a `pd.DataFrame`. Each DataFrame is indexed by species or reaction ids (s_ids and _r_ids) corresponding to the indecies of the `species` and `reactions` tables.

The main approaches for this are either:
1. Adding information during network creation. This is generally how reaction-centric information such as the STRING weights will be passed. 
2. Directly add species or reaction data joining data based on systematic identifiers stored in `s_Identifiers` or `r_Identifiers` attributes.

## Passing information to graphs

To apply network-based methods we generally want to map results onto either vertex or edge attributes. For edges this can involve passing information which can be used for weighting connections (to favor certain sources or weight based on quantitative evidence scores.) For vertices, adding attributes supports either visualization or inference approaches such as network propagation.

We propagate this information by using an `entity_weights` dictionary which specifies both the values we should pluck out of entity data table (as a dictionary key plus a pd.DataFrame column) but also how we can combine these values. This would allow us to combine species_data which may be from different sets of biomolecules (such as proteomics and metabolomics), or to weight edges derived from multiple sources which may be weighted in different ways (or possess no evidence scores at all).

# Demos

## Adding Data to Pathways

### During construction

You can create `SBML_dfs` objects in multiple ways:
1. translating results from an .sbml file
2. direct creation from a list of component pd.DataFrames (species, compartmentalized_species, compartments, reactions, and reaction_species)
3. using the edgelist format to specify pairwise relationships with minimal annotations

Of these, the edgelist format supports directly passing edge attributes into `reactions_data`. Basically, when defining edges all of the columns which are not required variables will be added to reactions_data.

Some example functions which pass attributes during creation are yeast.convert_idea_kinetics_to_sbml_dfs() and string.convert_string_to_sbml_dfs(). As a quick demo:

In [11]:
import logging
logger = logging.getLogger()
logger.setLevel("INFO")

import tutorial_utils
config = tutorial_utils.CprConfig("config.yaml", "downloading_pathway_data")

In [12]:
import pandas as pd

from napistu import sbml_dfs_core
from napistu import source
from napistu import identifiers
from napistu import utils

# setup compartments (just treat this as uncompartmentalized for now)
compartments_df = sbml_dfs_core._stub_compartments()

# Per convention unaggregated models receive an empty source
interaction_source = source.Source(init=True)

In [13]:
def _get_example_edgelist_inputs():
    interaction_edgelist = pd.DataFrame(
        [
            {
                "upstream_name": "A",
                "downstream_name": "B",
                "upstream_compartment": "cellular_component",
                "downstream_compartment": "cellular_component",
                "r_name": "A -> B",
                "sbo_term": "SBO:0000020",
                "r_Identifiers": identifiers.Identifiers(
                    [
                        identifiers.format_uri(
                            "http://www.ncbi.nlm.nih.gov/pubmed/10604467",
                            "BQB_IS_DESCRIBED_BY",
                        )
                    ]
                ),
                "r_isreversible": False,
                "rxn_attr_1": "foo",
                "rxn_attr_2": 1,
            },
            {
                "upstream_name": "A",
                "downstream_name": "C",
                "upstream_compartment": "cellular_component",
                "downstream_compartment": "cellular_component",
                "r_name": "A -> C",
                "sbo_term": "SBO:0000459",
                "r_Identifiers": identifiers.Identifiers(
                    [
                        identifiers.format_uri(
                            "http://www.ncbi.nlm.nih.gov/pubmed/10604467",
                            "BQB_IS_DESCRIBED_BY",
                        )
                    ]
                ),
                "r_isreversible": False,
                "rxn_attr_1": "bar",
                "rxn_attr_2": 2,
            },
        ],
        index=[0, 1],
    )

    species_df = pd.DataFrame(
        [
            {
                "s_name": "A",
                "s_Identifiers": identifiers.Identifiers(
                    [
                        identifiers.format_uri(
                            "http://www.ensembl.org/Homo_sapiens/geneview?gene=ENSG00000153094",
                            "BQB_IS",
                        )
                    ]
                ),
                "spec_attr": 2,
            },
            {
                "s_name": "B",
                "s_Identifiers": identifiers.Identifiers(
                    [
                        identifiers.format_uri(
                            "https://purl.uniprot.org/uniprot/Q557I5", "BQB_IS"
                        )
                    ]
                ),
                "spec_attr": 5,
            },
            {
                "s_name": "C",
                "s_Identifiers": identifiers.Identifiers(
                    [
                        identifiers.format_uri(
                            "http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:37136",
                            "BQB_IS",
                        )
                    ]
                ),
                "spec_attr": 1,
            },
        ],
        index=[0, 1, 2],
    )

    compartments_df = sbml_dfs_core._stub_compartments()

    interaction_source = source.Source(init=True)

    return (interaction_edgelist, species_df, compartments_df, interaction_source)


(
    interaction_edgelist,
    species_df,
    compartments_df,
    interaction_source,
) = _get_example_edgelist_inputs()

sbml_dfs = sbml_dfs_core.sbml_dfs_from_edgelist(
    interaction_edgelist,
    species_df,
    compartments_df,
    interaction_source,
    keep_species_data="data",
    keep_reactions_data="data",
)

#### Mounted species data 

In [14]:
utils.style_df(sbml_dfs.reactions_data["data"])

,rxn_attr_1,rxn_attr_2
r_id,,
R00000000,foo,1
R00000001,bar,2


#### Mounted reaction data

In [15]:
utils.style_df(sbml_dfs.species_data["data"])

,spec_attr
s_id,
S00000000,2
S00000001,5
S00000002,1


## Adding entity data to an existing `SBML_dfs` object

To add reaction- or species-level data to an existing `SBML_dfs` object we can create an appropriate pd.DataFrame and directly add it to the object. As with all `species_data` or `reactions_data` entries this table must be indexed by the models species or reaction ids. Because of this, the challenge in merging results determining which species in our model match entries in the to-be-added entity data. To provide some guidance on this we will consider a couple of cases: matching by names, and matching by standard identifiers.

### Matching by names

Matching by names or symbols is generally not a good idea because there is a many-to-many relationship between many genes and symbols. Still, lots of people do use symbols, and this is a simple case which shows how easy it is to add entity data once we've matched it to existing pathway species or reactions.


In [16]:
new_species_data = sbml_dfs.species[0:2].assign(new_data=2)[["new_data"]]

new_reactions_data = pd.DataFrame(
    [
        {"r_id": sbml_dfs.reactions.index[0], "new_data": 2},
        {"r_id": sbml_dfs.reactions.index[1], "new_data": 3},
    ]
).set_index("r_id")

sbml_dfs.add_species_data("new_data", new_species_data)
sbml_dfs.add_reactions_data("new_data", new_reactions_data)

utils.style_df(sbml_dfs.species_data["new_data"])

,new_data
s_id,
S00000000,2
S00000001,2


### Matching by identifiers

Generally we will be trying to add molecular data to a network which is associated with one or more systematic ontologies. A nice way to do this is using `mechanism_matching.features_to_pathway_species()`. This function will compare a table containing all species or reactions identifiers in the pathway model to a set of query features to create a lookup table of query identifiers to pathway ids.

In [17]:
from napistu import mechanism_matching

# export identifiers from pathway
species_identifiers = sbml_dfs.get_identifiers("species")

feature_annotations = pd.DataFrame(
    [
        {"identifier": "ENSG00000153094", "expression": 1000},
        {"identifier": "ENSG0000000000", "expression": 50},
    ],
    index=[0, 1],
)

updated_species_data = mechanism_matching.features_to_pathway_species(
    feature_annotations,
    species_identifiers,
    ontologies={"ensembl_gene"},
    feature_id_var="identifier",
)[["s_id", "expression"]].set_index("s_id")

sbml_dfs.add_species_data("newest_data", updated_species_data)

#### Mounted species data

In [18]:
sbml_dfs.species_data

{'data':            spec_attr
 s_id                
 S00000000          2
 S00000001          5
 S00000002          1,
 'new_data':            new_data
 s_id               
 S00000000         2
 S00000001         2,
 'newest_data':            expression
 s_id                 
 S00000000        1000}

#### Mounted reaction data

In [19]:
sbml_dfs.reactions_data

{'data':           rxn_attr_1  rxn_attr_2
 r_id                            
 R00000000        foo           1
 R00000001        bar           2,
 'new_data':            new_data
 r_id               
 R00000000         2
 R00000001         3}

## Passing Information to Graphs 

Now that we have our data of interest tied to the appropriate species and reactions in our pathway we can carry this information forward as we translate the pathway representation into a graph of vertices connected by edges.

This process is controlled by the settings in the `reaction_graph_attrs` dictionary which specifies the variables which should pulled out of `species_data` or `reactions_data` and can also be used to specify how the graph should be weighted. A real-world example of this can be found in [calcification_causality.ipynb](https://github.com/calico/discovery/blob/main/projects/calcification/calcification_causality/calcification_causality.ipynb).


In [20]:
from napistu.network import net_create

reaction_graph_attrs = {
    "reactions": {
        "reaction_wts": {"table": "data", "variable": "rxn_attr_1", "trans": "identity"}
    },
    "species": {
        "species_var1": {
            "table": "data",
            "variable": "spec_attr",
            "trans": "string_inv",
        },
        "species_var2": {
            "table": "newest_data",
            "variable": "expression",
            "trans": "identity",
        },
    },
}

graph_w_annotations = net_create.create_napistu_graph(
    sbml_dfs,
    reaction_graph_attrs,
    directed=True,
    graph_type="regulatory"
)

INFO:napistu.network.net_create:Organizing all network nodes (compartmentalized species and reactions)
INFO:napistu.network.net_create:Formatting edges as a regulatory graph
INFO:napistu.network.net_create:Formatting 4 reactions species as tiered edges.
INFO:napistu.network.net_create:Adding additional attributes to edges, e.g., # of children and parents.
INFO:napistu.network.net_create:Done preparing regulatory graph
INFO:napistu.network.net_create:Adding reversibility and other meta-data from reactions_data
INFO:napistu.network.net_create:Creating reverse reactions for reversible reactions on a directed graph
INFO:napistu.network.net_create:Formatting cpr_graph output


### Graph vertices (with data)

In [21]:
utils.style_df(graph_w_annotations.get_vertex_dataframe())

,name,node_name,node_type
vertex ID,,,
0,SC00000000,A [cellular_component],species
1,SC00000001,B [cellular_component],species
2,SC00000002,C [cellular_component],species
3,R00000000,A -> B,reaction
4,R00000001,A -> C,reaction


### Graph edges (with data)

In [22]:
utils.style_df(graph_w_annotations.get_edge_dataframe())

,source,target,from,to,sbo_term,stoichiometry,sc_parents,sc_degree,species_type,r_id,sc_children,r_isreversible,reaction_wts,direction
edge ID,,,,,,,,,,,,,,
0,3,1,R00000000,SC00000001,SBO:0000011,1.000,1.000,1.000,protein,R00000000,0.000,False,foo,forward
1,4,2,R00000001,SC00000002,SBO:0000011,1.000,1.000,1.000,metabolite,R00000001,0.000,False,bar,forward
2,0,3,SC00000000,R00000000,SBO:0000020,0.000,0.000,2.000,protein,R00000000,2.000,False,foo,forward
3,0,4,SC00000000,R00000001,SBO:0000459,0.000,0.000,2.000,protein,R00000001,2.000,False,bar,forward
